# Домашнее задание

In [1]:
from abc import ABC, abstractmethod
import logging

logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)

handler = logging.StreamHandler()
FORMAT = '%(asctime)s %(levelname)-8s %(message)s'
formatter = logging.Formatter(fmt=FORMAT)
handler.setFormatter(formatter)
logger.addHandler(handler)


class Base(ABC):
    objects: set

    def __init__(self, *args, **kwargs):
        self.objects.add(self)

    @abstractmethod
    def validate(self):
        pass

    @abstractmethod
    def log(self):
        pass

In [3]:
class Group(Base):
    objects = set()
    
    def __init__(self, name, num):
        self.name = name
        self.num = num
        if self.validate():
            super().__init__()
            self.log()

    def validate(self) -> bool:
        for g in Group.objects:
            if g.name == self.name or g.num == self.num:
                logger.error('Такая группа уже есть')
                return False
        return True
        
    def p(self, c):
        print(self.name, self.num)

    def __str__(self):
        return f'{self.num}: {self.name}'
        
    def __repr__(self):
        return str(self)
        
    def log(self):
        logger.info(f'Добавлена запись {self}')

In [5]:
class Student(Base):
    objects = set()
    
    def __init__(self, first_name, last_name, group):
        self.first_name = first_name
        self.last_name = last_name
        self.group = group
        if self.validate():
            super().__init__()
            self.log()

    def validate(self) -> bool:
        for s in self.objects:
            if s.first_name == self.first_name and s.last_name == self.last_name and s.group == self.group:
                logger.error('Такой студент уже есть')
                return False
        return True

    def __str__(self):
        return f'{self.first_name} {self.last_name}: {self.group}'
        
    def __repr__(self):
        return f'{self.first_name} {self.last_name}: {self.group}'
        
    def log(self):
        logger.info(f'Добавлена запись {self}')

In [7]:
from datetime import date

class Mark(Base):
    objects = set()
    def __init__(self, student, subject, mark):
        self.student = student
        self.subject = subject
        self.mark = mark
        self.data = date.today()
        if self.validate():
            super().__init__()
            self.log()
    
    def validate(self) -> bool:
        if self.mark < 2 or self.mark > 5:
            logger.error('Неправильная оценка')
            return False
        return True

    def __str__(self):
        return f'{self.student}, {self.subject}: {self.mark}'
        
    def __repr__(self):
        return f'{self.student}, {self.subject}: {self.mark}'
        
    def log(self):
        logger.info(f'Добавлена запись {self}')

In [9]:
import random
class Journal():
    def __init__(self):
        self.marks = []

    def __str__(self):
        return f'{self.marks}'
        
    def __repr__(self):
        return f'{self.marks}'
        
    def log(self):
        logger.info(f'Добавлена запись {self}')
        
    def add_mark(self, mark):
        self.marks.append(mark)

    def get_group(self, group_num, subject):
        l = [mark for mark in self.marks if mark.student.group.num == group_num and mark.subject == subject]
        l.sort(key=lambda x: (x.student.group.num, x.student.last_name, x.subject))
        return l

    def get_student(self, first_name: str, last_name: str) -> list[Mark]:
        l = [mark for mark in self.marks if mark.student.first_name == first_name and mark.student.last_name == last_name]
        l.sort(key=lambda x: x.subject)
        return l

    def avg(self, 
            subject: str = None, 
            first_name: str = None, 
            last_name: str = None, 
            group_num: int = None,
           ):
        l = [mark for mark in self.marks if (mark.student.group.num == group_num or group_num == None) and (mark.subject == subject or subject == None) and (mark.student.first_name == first_name or first_name == None) and (mark.student.last_name == last_name or last_name == None)]
        if not l:
            logger.warning('Нет оценок для данного студента по указанному предмету')
            return
        s = sum(mark.mark for mark in l)
        s = s / len(l)
        return s

    def get_5(self,
              subject: str = None, 
              group_num: int = None,
             ) -> list[Student] | None:
        l = []
        for mark in self.marks:
            if (mark.student.group.num == group_num or group_num == None) and (mark.subject == subject or subject == None):
                a = self.avg(subject, mark.student.first_name, mark.student.last_name, group_num)
                if a>=4.5 and not mark.student in l:
                    l.append(mark.student)
        if not l:
            logger.warning('Нет отличников')
            return
        return l

    def get_rand(self,
                 subject: str = None, 
                 group_num: int = None,
                ) -> Student | None:
        l = [mark for mark in self.marks if (mark.student.group.num == group_num or group_num == None) and (mark.subject == subject or subject == None)]
        v = [(5 - self.avg(subject, mark.student.first_name, mark.student.last_name, group_num)) for mark in l]
        s = random.choices(l, v, k=1)
        return s
    
    @property
    def count(self):
        return len(self.marks)

In [11]:
# 1. логирование ERROR
# 2. валидация в группе
# 3. Mark - self.date
# 4. Сделать опциональным фильтры в avg()
# 5. метод, чтобы вывести отличников (по группе, по предмету)
# 6. метод, вызывающий рандомно студента по предмету и группе

from datetime import date
date.today()

datetime.date(2024, 12, 22)

In [13]:
group1 = Group("Группа 2", 1)
group2 = Group("Группа 1", 2)

2024-12-22 10:42:16,933 INFO     Добавлена запись 1: Группа 2
2024-12-22 10:42:16,934 INFO     Добавлена запись 2: Группа 1


In [15]:
group3 = Group("Группа 2", 1)

2024-12-22 10:42:17,421 ERROR    Такая группа уже есть


In [17]:
student1 = Student("Иван", "Иванов", group1)
student2 = Student("Иван", "Петров", group1)
student3 = Student("Сергей", "Сергеев", group2)

2024-12-22 10:42:17,869 INFO     Добавлена запись Иван Иванов: 1: Группа 2
2024-12-22 10:42:17,869 INFO     Добавлена запись Иван Петров: 1: Группа 2
2024-12-22 10:42:17,871 INFO     Добавлена запись Сергей Сергеев: 2: Группа 1


In [19]:
journal = Journal()
journal.add_mark(Mark(student1, "Математика", 4))
journal.add_mark(Mark(student1, "Физика", 5))
journal.add_mark(Mark(student2, "Математика", 3))
journal.add_mark(Mark(student3, "Физика", 4))
journal.add_mark(Mark(student1, "Математика", 3))
journal.add_mark(Mark(student1, "Математика", 4))
journal.add_mark(Mark(student1, "Математика", 4))
journal.add_mark(Mark(student1, "Математика", 5))

2024-12-22 10:42:18,354 INFO     Добавлена запись Иван Иванов: 1: Группа 2, Математика: 4
2024-12-22 10:42:18,354 INFO     Добавлена запись Иван Иванов: 1: Группа 2, Физика: 5
2024-12-22 10:42:18,354 INFO     Добавлена запись Иван Петров: 1: Группа 2, Математика: 3
2024-12-22 10:42:18,355 INFO     Добавлена запись Сергей Сергеев: 2: Группа 1, Физика: 4
2024-12-22 10:42:18,356 INFO     Добавлена запись Иван Иванов: 1: Группа 2, Математика: 3
2024-12-22 10:42:18,356 INFO     Добавлена запись Иван Иванов: 1: Группа 2, Математика: 4
2024-12-22 10:42:18,356 INFO     Добавлена запись Иван Иванов: 1: Группа 2, Математика: 4
2024-12-22 10:42:18,357 INFO     Добавлена запись Иван Иванов: 1: Группа 2, Математика: 5


In [21]:
b = journal.get_student("Иван", "Иванов")
for mark in b:
    print(f"{mark.subject}: {mark.mark}")

Математика: 4
Математика: 3
Математика: 4
Математика: 4
Математика: 5
Физика: 5


In [23]:
s = journal.avg("Математика", "Иван", "Иванов", 1)
s

4.0

In [25]:
a = journal.get_group(1, "Математика")
for mark in a:
    print(f"{mark.student.first_name} {mark.student.last_name}: {mark.mark}")

Иван Иванов: 4
Иван Иванов: 3
Иван Иванов: 4
Иван Иванов: 4
Иван Иванов: 5
Иван Петров: 3


In [27]:
s = journal.avg("Математика", None, None, 1)
s

3.8333333333333335

In [29]:
journal.get_5()

2024-12-22 10:42:20,747 WARNING  Нет отличников


In [31]:
student4 = Student("Алина", "Редькина", group2)

2024-12-22 10:42:21,202 INFO     Добавлена запись Алина Редькина: 2: Группа 1


In [33]:
journal = Journal()
journal.add_mark(Mark(student4, "Математика", 4))
journal.add_mark(Mark(student4, "Физика", 5))
journal.add_mark(Mark(student4, "Математика", 5))
journal.add_mark(Mark(student4, "Физика", 5))
journal.add_mark(Mark(student4, "Математика", 5))
journal.add_mark(Mark(student4, "Математика", 5))
journal.add_mark(Mark(student4, "Математика", 4))
journal.add_mark(Mark(student4, "Математика", 5))

2024-12-22 10:42:21,684 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Математика: 4
2024-12-22 10:42:21,686 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Физика: 5
2024-12-22 10:42:21,687 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Математика: 5
2024-12-22 10:42:21,687 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Физика: 5
2024-12-22 10:42:21,689 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Математика: 5
2024-12-22 10:42:21,689 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Математика: 5
2024-12-22 10:42:21,689 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Математика: 4
2024-12-22 10:42:21,690 INFO     Добавлена запись Алина Редькина: 2: Группа 1, Математика: 5


In [35]:
s = journal.avg("Математика", "Алина", "Редькина")
s

4.666666666666667

In [37]:
s = journal.avg("Физика", "Алина", "Редькина")
s

5.0

In [39]:
s = journal.avg(None, "Алина", "Редькина")
s

4.75

In [41]:
journal.get_5()

[Алина Редькина: 2: Группа 1]

In [43]:
journal.get_rand()

[Алина Редькина: 2: Группа 1, Математика: 4]

In [45]:
journal.get_rand()

[Алина Редькина: 2: Группа 1, Физика: 5]

In [8]:
def can_make_zero_even(n):
    # Получаем все четные числа до n
    even_numbers = [i for i in range(2, n + 1, 2)]

    # Проверяем все комбинации знаков для четных чисел
    from itertools import product
    for signs in product([-1, 1], repeat=len(even_numbers)):
        if sum(sign * number for sign, number in zip(signs, even_numbers)) == 0:
            return True
    return False
for n in range(1, 101):
    if can_make_zero_even(n):
        print(n)

1
6
7
8
9
14
15
16
17
22
23
24
25
30
31
32
33


KeyboardInterrupt: 